#### The scope of this project is to apply LSTM to predict the returns for 7 high-growth assets. Most of these stocks are tech stock that have extremely high returns during the bull run 2016-2025 so their average returns are unrealisticly high. LSTM will shrink these returns.

In [1]:
import yfinance as yf 
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [2]:
# The 7 high-growth assets
tickers=["HWM", "NVDA", "MSI", "AMZN", "MA", "TSLA", "ALB"]

In [3]:
prices = yf.download(tickers, start="2016-11-01", end="2025-09-02",interval="1mo")["Close"]

/var/folders/qm/71bzq9451kqf888zy5kpswfr0000gn/T/ipykernel_66128/2425981317.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  prices = yf.download(tickers, start="2016-11-01", end="2025-09-02",interval="1mo")["Close"]
[*********************100%***********************]  7 of 7 completed


In [4]:
returns = np.log(prices).diff().dropna() #returns of each asset

---
# Base model

This model is simply an LSTM model using historical returns to predict future return. Simply a time series model

In [5]:
def make_sequences(X_df, y_df, lookback=12):
    X_list, y_list = [], []
    for i in range(len(X_df)-lookback):
        X_window = X_df.iloc[i:i+lookback].values     # (L, n_features)
        y_next   = y_df.iloc[i+lookback].values       # (n_tickers,)
        X_list.append(X_window)
        y_list.append(y_next)
    return np.array(X_list, dtype="float32"), np.array(y_list, dtype="float32")


In [6]:
X_base = returns.copy()
y_base = returns.shift(-1).dropna()
X_base = X_base.loc[y_base.index]  # align or, in this case, drop the last month because there's no y for that month

In [7]:
# base model: pure historical prices
Xb, yb = make_sequences(X_base, y_base)

In [8]:

def train_val_split(X, y, val_size=0.2):
    split = int(len(X)*(1-val_size))
    return X[:split], X[split:], y[:split], y[split:]

Xtr_base, Xval_base, ytr_base, yval_base = train_val_split(Xb, yb)

# scale
sc_X, sc_y = StandardScaler(), StandardScaler()
Xtr_base = sc_X.fit_transform(Xtr_base.reshape(-1, Xtr_base.shape[2])).reshape(Xtr_base.shape)
Xval_base = sc_X.transform(Xval_base.reshape(-1, Xval_base.shape[2])).reshape(Xval_base.shape)

ytr_base = sc_y.fit_transform(ytr_base)   # (Ntr, 7)
yval_base = sc_y.transform(yval_base)     # (Nval, 7)


In [9]:
def build_model(input_shape):
    model = Sequential([
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32, return_sequences=False),
        Dense(16, activation="relu"),
        Dense(7)   # target return for 7 assets
    ])
    model.compile(optimizer="adam", loss="mse")
    return model

model = build_model((Xtr_base.shape[1], Xtr_base.shape[2])) #Model

/Users/banguyen/miniconda3/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 12, 64)         │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 12, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,495 (123.03 KB)

 Trainable params: 31,495 (123.03 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Training
es = EarlyStopping(patience=10, restore_best_weights=True)
rlr = ReduceLROnPlateau(patience=5, factor=0.5, min_lr=1e-5)

base = model.fit(
    Xtr_base, ytr_base,
    validation_data=(Xval_base, yval_base),
    epochs=200,
    batch_size=8,
    shuffle=False,  # keep time order
    callbacks=[es, rlr],
    verbose=1
)

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - loss: 1.0045 - val_loss: 0.5391 - learning_rate: 0.0010
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9894 - val_loss: 0.5363 - learning_rate: 0.0010
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9839 - val_loss: 0.5366 - learning_rate: 0.0010
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9745 - val_loss: 0.5386 - learning_rate: 0.0010
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9660 - val_loss: 0.5416 - learning_rate: 0.0010
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9554 - val_loss: 0.5440 - learning_rate: 0.0010
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9463 - val_loss: 0.5493 - learning_rate: 0.0010
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9403 - val_loss: 0.5532 - learning_rate: 5.0000e-04
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9348 - val_loss: 0.5583 - learning_rate: 5.0000e-04
Epoch 10/

Our model stops after 12 epoches with very small learning rates for each epoch and the loss is only slightly reduced

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Predict on validation X (scaled) → invert y scaling
yval_pred_scaled = model.predict(Xval_base, verbose=0)          # shape (Nval, 7)
yval_pred = sc_y.inverse_transform(yval_pred_scaled)        # real returns
yval_true = sc_y.inverse_transform(yval_base)                    # real returns

In [13]:
per_ticker = {}
for j, t in enumerate(tickers):
    rmse_j = np.sqrt(mean_squared_error(yval_true[:, j], yval_pred[:, j]))
    mae_j  = mean_absolute_error(yval_true[:, j], yval_pred[:, j])
    r2_j   = r2_score(yval_true[:, j], yval_pred[:, j])
    dir_j  = (np.sign(yval_pred[:, j]) == np.sign(yval_true[:, j])).mean()
    per_ticker[t] = dict(RMSE=rmse_j, MAE=mae_j, R2=r2_j, DirAcc=dir_j)
per_ticker

{'HWM': {'RMSE': np.float64(0.11855187891943722),
  'MAE': 0.09139399975538254,
  'R2': -0.03265273571014404,
  'DirAcc': np.float64(0.5789473684210527)},
 'NVDA': {'RMSE': np.float64(0.06452652696293114),
  'MAE': 0.05436856672167778,
  'R2': -0.00507962703704834,
  'DirAcc': np.float64(0.631578947368421)},
 'MSI': {'RMSE': np.float64(0.10450831404693452),
  'MAE': 0.08156838268041611,
  'R2': -0.14600491523742676,
  'DirAcc': np.float64(0.631578947368421)},
 'AMZN': {'RMSE': np.float64(0.03841248388995703),
  'MAE': 0.03206741809844971,
  'R2': -0.016925811767578125,
  'DirAcc': np.float64(0.5789473684210527)},
 'MA': {'RMSE': np.float64(0.052758348278656275),
  'MAE': 0.041712310165166855,
  'R2': -0.001048445701599121,
  'DirAcc': np.float64(0.7368421052631579)},
 'TSLA': {'RMSE': np.float64(0.10130987476534017),
  'MAE': 0.08297139406204224,
  'R2': 0.002702772617340088,
  'DirAcc': np.float64(0.631578947368421)},
 'ALB': {'RMSE': np.float64(0.14692626423716182),
  'MAE': 0.117060

This model does a slightly-better-than-random. It's not good enough to replace the mean baseline.

---
## Indicators

This model is similar to the first one, but we will add some indicators, which are tied directly to the prices, so maybe a bit different but the base of the model is still the prices.

In [14]:
import ta

In [15]:
features = []
for t in tickers:
    ind = pd.DataFrame(index=prices.index)
    ind[f"{t}_ret"] = returns[t]
    
    # Example indicators
    ind[f"{t}_rsi"] = ta.momentum.RSIIndicator(prices[t]).rsi()
    ind[f"{t}_sma20"] = ta.trend.SMAIndicator(prices[t], window=20).sma_indicator()
    ind[f"{t}_sma50"] = ta.trend.SMAIndicator(prices[t], window=50).sma_indicator()
    ind[f"{t}_macd"] = ta.trend.MACD(prices[t]).macd()
    ind[f"{t}_bb_high"] = ta.volatility.BollingerBands(prices[t]).bollinger_hband()
    ind[f"{t}_bb_low"]  = ta.volatility.BollingerBands(prices[t]).bollinger_lband()
    
    features.append(ind)

In [16]:
X_full = pd.concat(features, axis=1).dropna()
y_full = returns.loc[X_full.index]

In [17]:
X_ind, y_ind = make_sequences(X_full, y_full)

In [18]:
# ====== 6. Train/validation split ======
split = int(len(X_ind)*0.8)
Xtr_ind, Xval_ind = X_ind[:split], X_ind[split:]
ytr_ind, yval_ind = y_ind[:split], y_ind[split:]


In [19]:
# ====== 7. Scale features ======
Xtr_ind = sc_X.fit_transform(Xtr_ind.reshape(-1, Xtr_ind.shape[2])).reshape(Xtr_ind.shape)
Xval_ind = sc_X.transform(Xval_ind.reshape(-1, Xval_ind.shape[2])).reshape(Xval_ind.shape)
ytr_ind = sc_y.fit_transform(ytr_ind)
yval_ind = sc_y.transform(yval_ind)

In [20]:
# ====== 8. Define multi-output LSTM ======
n_features = X_ind.shape[2]
n_assets = y_ind.shape[1]
seq_len = X_ind.shape[1]
print("n_features:", n_features, "; n_assets:", n_assets, "; seq_len:", seq_len)

n_features: 49 ; n_assets: 7 ; seq_len: 12


In [21]:
model2 = build_model((Xtr_ind.shape[1], Xtr_ind.shape[2])) #Model 2
history_ind = model2.fit(
    Xtr_ind, ytr_ind,
    validation_data=(Xval_ind, yval_ind),
    epochs=200,
    batch_size=8,
    shuffle=False,
    callbacks=[es, rlr],
    verbose=1
)

Epoch 1/200


/Users/banguyen/miniconda3/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - loss: 1.0179 - val_loss: 0.6436 - learning_rate: 0.0010
Epoch 2/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.9835 - val_loss: 0.6467 - learning_rate: 0.0010
Epoch 3/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.9607 - val_loss: 0.6584 - learning_rate: 0.0010
Epoch 4/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9439 - val_loss: 0.6744 - learning_rate: 0.0010
Epoch 5/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9438 - val_loss: 0.6879 - learning_rate: 0.0010
Epoch 6/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9361 - val_loss: 0.6928 - learning_rate: 5.0000e-04
Epoch 7/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9278 - val_loss: 0.6955 - learning_rate: 5.0000e-04
Epoch 8/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9192 - val_loss: 0.6969 - learning_rate: 5.0000e-04
Epoch 9/200
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.9153 - val_loss: 0.7001 - learning_rate: 5.0000e-04
Epoch 10/200
5/5 ━━━━━━━

In [22]:
# ====== 11. Predict & evaluate ======
yval_pred_ind = model2.predict(Xval_ind)
yval_pred_ind = sc_y.inverse_transform(yval_pred_ind)
yval_true_ind = sc_y.inverse_transform(yval_ind)

results = {}
for i, ticker in enumerate(tickers):
    results[ticker] = {
        "RMSE": mean_squared_error(yval_true_ind[:,i], yval_pred_ind[:,i]),
        "MAE": mean_absolute_error(yval_true_ind[:,i], yval_pred_ind[:,i]),
        "R2": r2_score(yval_true_ind[:,i], yval_pred_ind[:,i]),
        "DirAcc": np.mean(np.sign(yval_true_ind[:,i]) == np.sign(yval_pred_ind[:,i]))
    }

print(results)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step
{'HWM': {'RMSE': 0.01740124262869358, 'MAE': 0.1021505743265152, 'R2': -0.011916875839233398, 'DirAcc': np.float64(0.7)}, 'NVDA': {'RMSE': 0.005388725083321333, 'MAE': 0.06562988460063934, 'R2': 0.017145216464996338, 'DirAcc': np.float64(0.4)}, 'MSI': {'RMSE': 0.0080042639747262, 'MAE': 0.07811928540468216, 'R2': -0.05521583557128906, 'DirAcc': np.float64(0.6)}, 'AMZN': {'RMSE': 0.0015459759160876274, 'MAE': 0.033508043736219406, 'R2': -0.020122766494750977, 'DirAcc': np.float64(0.5)}, 'MA': {'RMSE': 0.003343359101563692, 'MAE': 0.045809995383024216, 'R2': -0.5446550846099854, 'DirAcc': np.float64(0.6)}, 'TSLA': {'RMSE': 0.012073269113898277, 'MAE': 0.0941978469491005, 'R2': -0.024521350860595703, 'DirAcc': np.float64(0.5)}, 'ALB': {'RMSE': 0.022078534588217735, 'MAE': 0.11610134690999985, 'R2': -0.05424630641937256, 'DirAcc': np.float64(0.5)}}


This model's performance is no better than the first one, if not worse

Except for HWM the model is almost random.

--- 
# Relative assets (competitors & market movers)

We will train 7 seperated models to predict the returns for each assets separatedly with the same set of relative assets. 
Ten relative assets include:
1. SPY (S&P500): broad US market
2. QQQ: tech heavy, captures growth
3. XLK: technology sector (NVDA, TSLA, MA, MSI)
4. XLY: consumer discretionary (AMZN, TSLA)
5. XLF: financials (MA, MSI partly)
6. XLI: industrials (HWM, aerospace/defense)
7. XLB: materials (ALB)
8. SMH: semiconductors ETF (NVDA, indirectly TSLA/autonomous)
9. ITA: aerospace & defense (HWM)
10. LIT: lithium/battery ETF (ALB, TSLA)

In [23]:
r_a=["SPY","QQQ","XLK","XLY","XLI","XLF","XLB","SMH","ITA","LIT"]

In [24]:
relatives = yf.download(r_a, start="2016-11-01", end="2025-09-02",interval="1mo")["Close"]
ra_returns=relatives.pct_change().dropna()

/var/folders/qm/71bzq9451kqf888zy5kpswfr0000gn/T/ipykernel_66128/2914001459.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  relatives = yf.download(r_a, start="2016-11-01", end="2025-09-02",interval="1mo")["Close"]
[*********************100%***********************]  10 of 10 completed


In [25]:
X_ra, y_ra = make_sequences(ra_returns, returns)

In [26]:
print(X_ra.shape,y_ra.shape)

(94, 12, 10) (94, 7)


In [27]:
split = int(0.8*len(X_ra))
Xtr_ra, Xval_ra = X_ra[:split], X_ra[split:]
ytr_ra, yval_ra = y_ra[:split], y_ra[split:]

In [28]:
model_ra=build_model((X_ra.shape[1],X_ra.shape[2]))

/Users/banguyen/miniconda3/lib/python3.13/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [29]:
model_ra.fit(Xtr_ra, ytr_ra,
          validation_data=(Xval_ra, yval_ra),
          epochs=200, batch_size=8,
          callbacks=[es, rlr],
          verbose=1
)

Epoch 1/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - loss: 0.0165 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 2/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0163 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 3/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0161 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 4/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0160 - val_loss: 0.0093 - learning_rate: 0.0010
Epoch 5/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0160 - val_loss: 0.0095 - learning_rate: 0.0010
Epoch 6/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 0.0160 - val_loss: 0.0093 - learning_rate: 0.0010
Epoch 7/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0160 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 8/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0160 - val_loss: 0.0096 - learning_rate: 0.0010
Epoch 9/200
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0159 - val_loss: 0.0094 - learning_rate: 0.0010
Epoch 10/200
10/10

In [30]:
yval_pred_ra = model_ra.predict(Xval_ra)
def evaluate_multi(y_true, y_pred):
    n_assets = y_true.shape[1]
    results = {}
    
    for i in range(n_assets):
        results[f"Asset_{i+1}"] = {
            "RMSE": np.sqrt(mean_squared_error(y_true[:, i], y_pred[:, i])),
            "MAE": mean_absolute_error(y_true[:, i], y_pred[:, i]),
            "R2": r2_score(y_true[:, i], y_pred[:, i]),
            "DirAcc": np.mean(np.sign(y_true[:, i]) == np.sign(y_pred[:, i]))
        }
    return results

print(evaluate_multi(yval_ra, yval_pred_ra))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 213ms/step
{'Asset_1': {'RMSE': np.float64(0.12052497803340285), 'MAE': 0.09314992278814316, 'R2': -0.06731224060058594, 'DirAcc': np.float64(0.42105263157894735)}, 'Asset_2': {'RMSE': np.float64(0.0644192064508844), 'MAE': 0.05441601574420929, 'R2': -0.0017391443252563477, 'DirAcc': np.float64(0.631578947368421)}, 'Asset_3': {'RMSE': np.float64(0.10196750839031625), 'MAE': 0.08067941665649414, 'R2': -0.09095895290374756, 'DirAcc': np.float64(0.631578947368421)}, 'Asset_4': {'RMSE': np.float64(0.03802192792693993), 'MAE': 0.03198592737317085, 'R2': 0.003648042678833008, 'DirAcc': np.float64(0.5789473684210527)}, 'Asset_5': {'RMSE': np.float64(0.05375138623715002), 'MAE': 0.04291767627000809, 'R2': -0.03908729553222656, 'DirAcc': np.float64(0.7368421052631579)}, 'Asset_6': {'RMSE': np.float64(0.10201295124107705), 'MAE': 0.08346647024154663, 'R2': -0.011187553405761719, 'DirAcc': np.float64(0.631578947368421)}, 'Asset_7': {'RMSE': np.float64(0.146470558786506

This performance is somewhat similar to the first model, if not worse. We understand that stocks are random and historical prices are noise. For that reason, to even predict about 25% of the exact returns seems impossible. Still, we will incorporate these predictions to balance out the average returns.

In [31]:
lookback=12
last_window = returns[-lookback:].values   # shape (12, 7)
last_window = last_window.reshape(1, lookback, 7)  # add batch dim

# Step 3: Predict next month
next_pred_scaled = model.predict(last_window)  # shape (1, 7)
next_pred = sc_y.inverse_transform(next_pred_scaled)  # rescale

# Step 4: Iterative forecasting (e.g., 6 months ahead)
future_preds = []
window = last_window.copy()

for _ in range(6):  # forecast 6 months
    pred_scaled = model.predict(window)
    pred = sc_y.inverse_transform(pred_scaled)
    future_preds.append(pred.ravel())

    # roll the window forward: drop oldest, add prediction
    new_row = pred_scaled  # keep in scaled space for model input
    window = np.concatenate([window[:,1:,:], new_row.reshape(1,1,7)], axis=1)

future_preds = np.array(future_preds)  # shape (6, 7)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


In [33]:
[r.mean() for r in future_preds.T]

[np.float32(-0.023349298),
 np.float32(0.003958315),
 np.float32(0.04078467),
 np.float32(0.015256338),
 np.float32(0.020482888),
 np.float32(0.043185662),
 np.float32(-0.0031317333)]